In [ ]:
import pandas as pd
from apis_core.helper_functions.RDFparsers import GenericRDFParser

In [ ]:
file = 'data/ASBW Personen und Werke - orgName.csv'
col, _ = Collection.objects.get_or_create(
    name='ASBW'
)
default_domain = "https://schnitzler-briefe.acdh.oeaw.ac.at"

In [ ]:
inst_place_rel, _ = InstitutionPlaceRelation.objects.get_or_create(name='located in', name_reverse='hosts')
pla_pla_rel, _ = PlacePlaceRelation.objects.get_or_create(name='part of', name_reverse='has part')

In [ ]:
df = pd.read_csv(file).fillna('FALSE')
df.keys()

In [ ]:
for i, row in df.iterrows():
    if row['Titel'] == 'FALSE':
        pass
    else:
        legacy_id = "{}/A{}".format(default_domain, row['Nummer-laufend'])
        item_type, _ = InstitutionType.objects.get_or_create(name=row['Typ'])
        item = GenericRDFParser(legacy_id, 'Institution').get_or_create()
        item.name = row['Titel']
        item.kind = item_type
        item.collection.add(col)
        if row['Ort'] != 'FALSE':
            try:
                mainplace, _ = Place.objects.get_or_create(name=row['Ort'])
            except:
                mainplace, _ = Place.objects.get_or_create(name="CHECK-ME: {}".format(row['Ort']))
        else:
            mainplace = None
        if row['Bezirk'] != 'FALSE':
            try:
                bezirk, _ = Place.objects.get_or_create(name=row['Bezirk'])
            except:
                bezirk, _ = Place.objects.get_or_create(name="CHECK-ME: {}".format(row['Bezirk']))
            if mainplace:
                PlacePlace.objects.get_or_create(
                    related_placeA=bezirk,
                    related_placeB=mainplace,
                    relation_type=pla_pla_rel
                )
            mainplace = bezirk
        if mainplace:
            InstitutionPlace.objects.get_or_create(
                related_institution=item,
                related_place=mainplace,
                relation_type=inst_place_rel
            )
        
        item.save()

In [ ]:
Place.objects.all().delete()